## Genetski algoritem

Z genetskim algoritmom želiva ovreči spodnjo neenakost

$ Ra(G) \geq rad(G) -1$

In [1]:
def randic(graf):
    '''vrne randicev indeks za podan graf'''
    vsota = 0
    povezave = graf.edges()
    stopnje = graf.degree()
    for edge in povezave:
        u, v,_ = edge
        d_u = stopnje[u]
        d_v = stopnje[v]
        vsota += 1/((d_u * d_v)**(1/2))
    return(vsota)

In [2]:
def fitness(graf):
    '''Vrne vrednost naše neenakosti, če je negativna lema ne drži'''
    return randic(graf) - graf.radius() + 1

In [5]:
def tournament_selection(populacija, t):
    '''med t naključno izbranimi grafi izbere tistega z najmanšim fitnesom'''
    velikost_populacije = len(populacija)
    n = randint(0, velikost_populacije - 1)
    najbolsi = populacija[n]
    fitnes_najbolsi = fitness(najbolsi)
    for i in range(1, t + 1):
        n = randint(0, velikost_populacije - 1)
        naslednji = populacija[n]
        fitnes = fitness(naslednji)
        if fitnes < fitnes_najbolsi:
            najbolsi = naslednji
            fitnes_najbolsi = fitnes
    return najbolsi

In [4]:
def poisson(t = 1, lambd = 1/2):
    '''poissonova porazdelitev'''
    N = 0
    S = 0
    while S < t:
        N += 1
        S += expovariate(lambd)
    return N

In [4]:
def mutation(graf):
    '''mutira graf'''
    kopija = graf
    verjetnost = random()
    plus_povezave = poisson(lambd = 1/2)
    minus_povezave = poisson(lambd = 1/2)
    if verjetnost <= 1/3:
        for k in range(plus_povezave):
            a, b = kopija.random_vertex(), kopija.random_vertex()
            if a != b:
                kopija.add_edge(a, b)
    elif verjetnost > 1/3 and verjetnost <= 2/3:
        for k in range(minus_povezave):
            a, b = kopija.random_vertex(), kopija.random_vertex()
            kopija.delete_edge(a, b)
            if not kopija.is_connected():
                kopija.add_edge(a, b)
    elif verjetnost > 2/3:
        for k in range(plus_povezave):
            a, b = kopija.random_vertex(), kopija.random_vertex()
            if a != b:
                kopija.add_edge(a, b)
        for k in range(minus_povezave):
            a, b = kopija.random_vertex(), kopija.random_vertex()
            kopija.delete_edge(a, b)
            if not kopija.is_connected():
                kopija.add_edge(a, b)
    return kopija

In [3]:
def crossover(a, b):
    '''križa dva grafa med sabo'''
    n = len(a)
    while True:
        podgraf_a = a.random_subgraph(0.5)
        podgraf_b = b.random_subgraph(0.5)
        if len(podgraf_a.vertices()) + len(podgraf_b.vertices()) == n and len(podgraf_a.vertices()) >= 1 and len(podgraf_a.vertices()) < n and podgraf_a.is_connected() and podgraf_b.is_connected():
            podgraf_a.relabel()
            podgraf_b.relabel()
            potomec = podgraf_a.disjoint_union(podgraf_b)
            nove_povezave = poisson(lambd = log(n/2))
            for k in range(nove_povezave):
                u = podgraf_a.random_vertex()
                v = podgraf_b.random_vertex()
                potomec.add_edge((0, u), (1, v))
            potomec.relabel()
            break
    return potomec

In [11]:
def zacetna_populacija(velikost, n):
    '''naredi začetno populacijo kjer imajo grafi n vozlišč'''
    populacija = []
    trenutna_velikost = 0
    while trenutna_velikost < velikost:
        graf = graphs.RandomGNP(n, random())
        if graf.is_connected():
            populacija.append(graf)
            trenutna_velikost += 1
    return populacija

In [2]:
def min_fitness(seznam):
    '''v seznamu poišče graf z najmanšim fitnesom'''
    najbolsi = seznam[0]
    fitnes = fitness(najbolsi)
    for graf in seznam:
        fit = fitness(graf)
        if fit < fitnes:
            fitnes = fit
            najbolsi = graf
    return najbolsi

In [38]:
def nova_populacija(populacija,t):
    '''zgenerira z križanjem in mutacijo naredi novo populacijo'''
    nova_populacija = []
    velikost_nove = 0
    velikost_stare = len(populacija)
    while velikost_nove < velikost_stare:
        graf1 = tournament_selection(populacija, t)
        graf2 = tournament_selection(populacija, t)
        copy1 = graf1
        copy2 = graf2
        mutacija1 = mutation(graf1)
        mutacija2 = mutation(graf2)
        krizan_graf = crossover(mutacija1, mutacija2)
        hibrid = mutation(crossover(graf1, graf2))
        optimalen_graf = min_fitness([krizan_graf, hibrid, mutacija1, mutacija2, copy1, copy2])
        nova_populacija.append(optimalen_graf)
        velikost_nove += 1
    return nova_populacija

In [1]:
def genetic_algorithm(n, k, cas_izvajanja, t=4):
    populacija = zacetna_populacija(n, k)
    for i in range(cas_izvajanja):
        for graf in populacija:
            fitnes = fitness(graf)
            print(round(fitnes, 0))
            if fitnes < 0:
                return graf
        populacija = nova_populacija(populacija,t)
    return True

In [39]:
genetic_algorithm(13, 15, 40, 10)

6
6
7
7
7
6
6
7


6
5
6
7
5


5
4
5
4
4
4
4
5
5
4
4
4
5


5
3
5
4
5
4
3
5
3
4
4
5
3


4
4
3
4
2
2
3
4
4
4
5
4
4


4
4
4
4
2
4
3
3
4
2
4
4
3


4
4
4
4
2
5
3
4
4
4
4
3
4


5
5
3
5
5
4
3
2
4
4
4
4
4


5
5
5
4
4
5
2
5
3
4
5
5
4


5
4
5
5
4
4
4
4
3
3
4
4
2
